In [2]:
import csv
returns = []
lens = []
times = []
with open('TrainLog.monitor.csv', mode='r') as csvfile:
    next(csvfile)
    reader = csv.DictReader(csvfile)
    for row in reader:
        returns.append(float(row['r']))
        lens.append(float(row['l']))
        times.append(float(row['t']))

print(len(returns))

4000
